# SUDOKU

but this time we solve it with a sat solver

In [4]:
from pysat.formula import *
from pysat.solvers import Solver
import numpy as np

In [2]:
from z3 import *

Let's dive into the idea. What are the constraints? How do we define our domain? 
We will use a 9 x 9 x 9 boolean space representation of the grid. Every cell will be represented as a vectors. 
In that vector every value is 0 excepted the i-th value that will be 1 and represent the actual value of the cell.

Let's make the grid.

In [5]:
I = [i for i in range (3)] # rows
J = [j for j in range (3)] # columns
V = [v for v in range (3)] # values
grid = np.array([[[Atom(f'{i+1}_{j+1}_{v+1}') for v in V ]for j in J] for i in I]) 